In [19]:
import subprocess
import json

def run_vqe(theta1, theta2, theta3, term_type=0, n_ham=0, n_op=0, shots=1, target="ionq.simulator"):
    cmd = f"az quantum execute --target-id {target} -- --theta1 {theta1} --theta2 {theta2} --theta3 {theta3} --term-type {term_type} --n-ham {n_ham} --n-op {n_op} --shots {shots}"
    print(cmd)
    run_cmd = subprocess.run([cmd], check=True, shell=True, capture_output=True)
    result = run_cmd.stdout
    result = json.loads(run_cmd.stdout.lstrip(b"."))
    return result, run_cmd

In [2]:
!az quantum execute --target-id ionq.simulator -- --theta1 0.001 --theta2 -0.001 --theta3 0.001 --term-type 0 --n-ham 0 --n-op 0

Command group 'quantum' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus
....
{
  "Histogram": [
    "0",
    1.2503123960000002e-07,
    "1",
    0.9999998749687428
  ]
}


In [3]:
run_vqe(theta1=0.001, theta2=-0.001, theta3=0.001, shots=1)

({'Histogram': ['0', 1.2503123960000002e-07, '1', 0.9999998749687428]},
 CompletedProcess(args=['az quantum execute --target-id ionq.simulator -- --theta1 0.001 --theta2 -0.001 --theta3 0.001 --term-type 0 --n-ham 0 --n-op 0 --shots 1'], returncode=0, stdout=b'....\n{\n  "Histogram": [\n    "0",\n    1.2503123960000002e-07,\n    "1",\n    0.9999998749687428\n  ]\n}\n', stderr=b"WARNING: Command group 'quantum' is in preview and under development. Reference and support levels: https://aka.ms/CLI_refstatus\n"))

In [4]:
hamiltonian = (
    [
        ([0], [0.17120128499999998]),
        ([1], [0.17120128499999998]),
        ([2], [-0.222796536]),
        ([3], [-0.222796536])
    ],
    [
        ([0, 1], [0.1686232915]),
        ([0, 2], [0.12054614575]),
        ([0, 3], [0.16586802525]),
        ([1, 2], [0.16586802525]),
        ([1, 3], [0.12054614575]),
        ([2, 3], [0.1743495025])
    ],
    [],
    [
        ([0, 1, 2, 3], [0.0, -0.0453218795, 0.0, 0.0453218795])
    ]
)

In [29]:
target = "ionq.simulator"
results = []
shots = 100
def run_vqe_all(theta1, theta2, theta3, target, shots):
    results = []
    for term_type, ham_terms in enumerate(hamiltonian):
        for n_ham, term in enumerate(ham_terms):
            _, ops = term
            for n_op, op in enumerate(ops):
                try:
                    result = run_vqe(theta1, theta2, theta3, term_type, n_ham, n_op, shots, target)
                except Exception:
                    result = run_vqe(theta1, theta2, theta3, term_type, n_ham, n_op, shots, target)
                print(result[0]["Histogram"][1])
                results.append(result)
    return results

In [30]:
def estimate_energy(hamiltonian, results, offset=-0.09883444600000002):
    termExpectation = [a["Histogram"][1] for a, _ in results]
    coeff = [__c for t in hamiltonian for _t, _c in t for __c in _c]
    return sum([(2. * _t - 1.) * _c for _t, _c in zip(termExpectation, coeff)]) + offset;

In [8]:
%%time
results = run_vqe_all(theta1=0.001, theta2=-0.001, theta3=0.001, target="ionq.simulator", shots=100)
estimate_energy(hamiltonian, results)

CPU times: user 19.2 ms, sys: 29.1 ms, total: 48.3 ms
Wall time: 2min 16s


-1.1166854544785538

In [9]:
from scipy.optimize import minimize

def run_program(var_params, num_samples) -> float:
    # run parameterized quantum program for VQE algorithm
    theta1, theta2, theta3 = var_params
    print(var_params)
    results = run_vqe_all(theta1, theta2, theta3, target="ionq.simulator", shots=num_samples)
    result = estimate_energy(hamiltonian, results)
    print(result)
    return result
 
def VQE(initial_var_params, num_samples):
    """ Run VQE Optimization to find the optimal energy and the associated variational parameters """
 
    opt_result = minimize(run_program,
                          initial_var_params,
                          args=(num_samples,),
                          method="COBYLA",
                          tol=0.000001,
                          options={'disp': True, 'maxiter': 200,'rhobeg' : 0.05})
 
    return opt_result

In [10]:
%%time
VQE([0.001, -0.001, 0.001], 100)

[ 0.001 -0.001  0.001]
-1.1166854544785538
[ 0.051 -0.001  0.001]
-1.1165775932987279
[0.001 0.049 0.001]
-1.1165858946327716
[ 0.001 -0.001  0.051]
-1.1164293673470773
[-0.01727087 -0.01786469 -0.04237923]
-1.1164824857403035
[-0.00813544 -0.00943234 -0.02068961]
-1.116636958112764
[ 0.0130225  -0.00223994 -0.00218936]
-1.1166779233970976
[ 0.001       0.00482526 -0.0012647 ]
-1.1166844707346502
[0.00082206 0.00264376 0.01295581]


KeyboardInterrupt: 

In [23]:
%%time
results = run_vqe_all(theta1=0.00756478, theta2=-0.00122729, theta3=-0.11203378, target="ionq.simulator", shots=100)
estimate_energy(hamiltonian, results)

az quantum execute --target-id ionq.simulator -- --theta1 0.00756478 --theta2 -0.00122729 --theta3 -0.11203378 --term-type 0 --n-ham 0 --n-op 0 --shots 100
0.0007875959704208
az quantum execute --target-id ionq.simulator -- --theta1 0.00756478 --theta2 -0.00122729 --theta3 -0.11203378 --term-type 0 --n-ham 1 --n-op 0 --shots 100
0.0007845398117727001
az quantum execute --target-id ionq.simulator -- --theta1 0.00756478 --theta2 -0.00122729 --theta3 -0.11203378 --term-type 0 --n-ham 2 --n-op 0 --shots 100
0.9992121908717752
az quantum execute --target-id ionq.simulator -- --theta1 0.00756478 --theta2 -0.00122729 --theta3 -0.11203378 --term-type 0 --n-ham 3 --n-op 0 --shots 100
0.9992156733459956
az quantum execute --target-id ionq.simulator -- --theta1 0.00756478 --theta2 -0.00122729 --theta3 -0.11203378 --term-type 1 --n-ham 0 --n-op 0 --shots 100
0.9999963292463481
az quantum execute --target-id ionq.simulator -- --theta1 0.00756478 --theta2 -0.00122729 --theta3 -0.11203378 --term-type

-1.1154472558126027

In [24]:
%%time
results = run_vqe_all(theta1=0.00756478, theta2=-0.00122729, theta3=-0.11203378, target="ionq.simulator", shots=1000)
estimate_energy(hamiltonian, results)

az quantum execute --target-id ionq.simulator -- --theta1 0.00756478 --theta2 -0.00122729 --theta3 -0.11203378 --term-type 0 --n-ham 0 --n-op 0 --shots 1000
0.0007875959704208
az quantum execute --target-id ionq.simulator -- --theta1 0.00756478 --theta2 -0.00122729 --theta3 -0.11203378 --term-type 0 --n-ham 1 --n-op 0 --shots 1000
0.0007845398117727001
az quantum execute --target-id ionq.simulator -- --theta1 0.00756478 --theta2 -0.00122729 --theta3 -0.11203378 --term-type 0 --n-ham 2 --n-op 0 --shots 1000
0.9992121908717752
az quantum execute --target-id ionq.simulator -- --theta1 0.00756478 --theta2 -0.00122729 --theta3 -0.11203378 --term-type 0 --n-ham 3 --n-op 0 --shots 1000
0.9992156733459956
az quantum execute --target-id ionq.simulator -- --theta1 0.00756478 --theta2 -0.00122729 --theta3 -0.11203378 --term-type 1 --n-ham 0 --n-op 0 --shots 1000
0.9999963292463481
az quantum execute --target-id ionq.simulator -- --theta1 0.00756478 --theta2 -0.00122729 --theta3 -0.11203378 --term

-1.1154472558126027

In [26]:
%%time
results = run_vqe_all(theta1=0.00756478, theta2=-0.00122729, theta3=-0.11203378, target="ionq.qpu", shots=1000)
estimate_energy(hamiltonian, results)

az quantum execute --target-id ionq.qpu -- --theta1 0.00756478 --theta2 -0.00122729 --theta3 -0.11203378 --term-type 0 --n-ham 0 --n-op 0 --shots 1000
0.6350000000000001
az quantum execute --target-id ionq.qpu -- --theta1 0.00756478 --theta2 -0.00122729 --theta3 -0.11203378 --term-type 0 --n-ham 1 --n-op 0 --shots 1000
0.6560000000000002
az quantum execute --target-id ionq.qpu -- --theta1 0.00756478 --theta2 -0.00122729 --theta3 -0.11203378 --term-type 0 --n-ham 2 --n-op 0 --shots 1000
0.6250000000000001
az quantum execute --target-id ionq.qpu -- --theta1 0.00756478 --theta2 -0.00122729 --theta3 -0.11203378 --term-type 0 --n-ham 3 --n-op 0 --shots 1000
0.6350000000000001
az quantum execute --target-id ionq.qpu -- --theta1 0.00756478 --theta2 -0.00122729 --theta3 -0.11203378 --term-type 1 --n-ham 0 --n-op 0 --shots 1000
0.6970000000000002
az quantum execute --target-id ionq.qpu -- --theta1 0.00756478 --theta2 -0.00122729 --theta3 -0.11203378 --term-type 1 --n-ham 1 --n-op 0 --shots 1000

0.0038779860485001344

In [27]:
%%time
results = run_vqe_all(theta1=0.00756478, theta2=-0.00122729, theta3=-0.11203378, target="ionq.qpu", shots=10000)
estimate_energy(hamiltonian, results)

az quantum execute --target-id ionq.qpu -- --theta1 0.00756478 --theta2 -0.00122729 --theta3 -0.11203378 --term-type 0 --n-ham 0 --n-op 0 --shots 10000
0.5557
az quantum execute --target-id ionq.qpu -- --theta1 0.00756478 --theta2 -0.00122729 --theta3 -0.11203378 --term-type 0 --n-ham 1 --n-op 0 --shots 10000
0.5747999999999999
az quantum execute --target-id ionq.qpu -- --theta1 0.00756478 --theta2 -0.00122729 --theta3 -0.11203378 --term-type 0 --n-ham 2 --n-op 0 --shots 10000
0.6130999999999999
az quantum execute --target-id ionq.qpu -- --theta1 0.00756478 --theta2 -0.00122729 --theta3 -0.11203378 --term-type 0 --n-ham 3 --n-op 0 --shots 10000
0.6551000000000001
az quantum execute --target-id ionq.qpu -- --theta1 0.00756478 --theta2 -0.00122729 --theta3 -0.11203378 --term-type 1 --n-ham 0 --n-op 0 --shots 10000
0.6713
az quantum execute --target-id ionq.qpu -- --theta1 0.00756478 --theta2 -0.00122729 --theta3 -0.11203378 --term-type 1 --n-ham 1 --n-op 0 --shots 10000
0.6789
az quantum

CalledProcessError: Command '['az quantum execute --target-id ionq.qpu -- --theta1 0.00756478 --theta2 -0.00122729 --theta3 -0.11203378 --term-type 1 --n-ham 3 --n-op 0 --shots 10000']' returned non-zero exit status 1.

In [31]:
%%time
results = run_vqe_all(theta1=0.00756478, theta2=-0.00122729, theta3=-0.11203378, target="ionq.qpu", shots=10000)
estimate_energy(hamiltonian, results)

az quantum execute --target-id ionq.qpu -- --theta1 0.00756478 --theta2 -0.00122729 --theta3 -0.11203378 --term-type 0 --n-ham 0 --n-op 0 --shots 10000


KeyboardInterrupt: 

In [ ]:
%%time
VQE([0.001, -0.001, 0.001], 100)